<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

In this notebook the focus is on the calculation of the minimal set of Ifrs Variable given the import Format. 

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "5ImportScope-ToIfrsVar"

# Actual dependent variables

In [0]:
public interface ComputeIfrsVarsActuals : ActualToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Actual.Concat(AdvanceActual).Concat(OverdueActual)
                                                               .Concat(ActEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss);
}

# Cashflow dependent variables

In [0]:
public interface ComputeIfrsVarsCashflows : ActualToIfrsVariable, PvToIfrsVariable, RaToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable, NominalToIfrsVariable, DeferrableAmortizationFactorToIfrsVariable
{
    private IEnumerable<IfrsVariable> amortizationFactors => AmortizationFactor.Union(DeferrableAmFactor, EqualityComparer<IfrsVariable>.Instance);
    
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Identity.ValuationApproach switch {
        ValuationApproaches.VFA => Actual.Concat(ActEAForPremium).Concat(PvCurrent).Concat(CumulatedNominal).Concat(RaCurrent).Concat(amortizationFactors)
                                   .Concat(BeEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss),
        _ =>  Actual.Concat(ActEAForPremium).Concat(PvLocked).Concat(PvCurrent).Concat(CumulatedNominal).Concat(RaCurrent).Concat(RaLocked).Concat(amortizationFactors)
             .Concat(BeEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss),
    };
}

# Chashflows with Projections

In [0]:
public interface ComputeIfrsVarsCashflowsWithProjection : IScope<ImportIdentity, ImportStorage>{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Enumerable.Range(0,ProjectionCount)
                                                                    .SelectMany(x => GetScope<ComputeIfrsVarsCashflows>(Identity with { ProjectionPeriod = x} ).CalculatedIfrsVariables)
                                                                    .AggregateProjections()
                                                                    .Select(x => x with {Partition = GetStorage().TargetPartition});
}

# Opening dependent variables

In [0]:
public interface ComputeIfrsVarsOpenings : ActualToIfrsVariable, DeferrableToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => AdvanceActual.Concat(OverdueActual)
                                                                      .Concat(Deferrable).Concat(Csms).Concat(Loss);
}

# Compute 

In [0]:
public interface ComputeAllScopes: IScope<ImportIdentity, ImportStorage>
{
   IEnumerable<IfrsVariable> CalculatedIfrsVariables => GetStorage().ImportFormat switch {
            ImportFormats.Actual   => GetScope<ComputeIfrsVarsActuals>(Identity).CalculatedIfrsVariables,
            ImportFormats.Cashflow => GetScope<ComputeIfrsVarsCashflowsWithProjection>(Identity).CalculatedIfrsVariables,
            ImportFormats.Opening  => GetScope<ComputeIfrsVarsOpenings>(Identity).CalculatedIfrsVariables,
   };
}